In [26]:
import pandas as pd
import numpy as np

num_topics = 26
df = pd.read_csv(f"~/Desktop/R_js/data/gamma/pro{num_topics}_gamma.csv")
df['log_gamma'] = np.absolute(np.log(df.gamma))
display(df.head())
print(df.shape)

,document,topic,gamma,log_gamma
0,1,1,0.007911,4.839471
1,2,1,0.015577,4.161986
2,3,1,0.012643,4.370688
3,4,1,0.013582,4.299036
4,5,1,0.015417,4.172280


(874120, 4)


## Getting the documents Review Text to display on cluster

In [27]:
proFilePath = "~/Desktop/R_js/data/pro_doc_sampled.csv"
proData = pd.read_csv(proFilePath)

proData.head()

,Unnamed: 0,Ratings,Job_Status,Reviewed_Year,Review_Text,Px_Texts,Tknz_Texts,Doc_Length,Work_Life,Benefits,Job_Advancement,Management,Culture,Company_Index,Company_Name
0,0,2.0,1,2017,"hour lunch, friendly co-workers.",cowork friendli hour lunch friendli_cowork hou...,"['cowork', 'friendli', 'hour', 'lunch', 'frien...",9,3.0,2.0,2.0,1.0,3.0,1,Walmart
1,1,3.0,1,2015,"benefits, Medical, dental, myshare, 401k, stocks",benefit dental medic myshar stock,"['benefit', 'dental', 'medic', 'myshar', 'stock']",5,3.0,4.0,4.0,3.0,3.0,1,Walmart
2,2,2.0,0,2012,discount card . schedule 3 on 3 off,card discount schedul discount_card,"['card', 'discount', 'schedul', 'discount_card']",4,1.0,4.0,3.0,1.0,3.0,1,Walmart
3,3,4.0,0,2017,good break lengths and plenty of hours,break hour length plenti plenti_hour,"['break', 'hour', 'length', 'plenti', 'plenti_...",5,4.0,3.0,3.0,4.0,4.0,1,Walmart
4,4,1.0,1,2016,My pay was good and my schedule allowed me to ...,allow job pay schedul pay_schedul,"['allow', 'job', 'pay', 'schedul', 'pay_schedul']",5,3.0,3.0,1.0,1.0,1.0,1,Walmart


In [28]:
doc_removed = np.asarray([1139, 1991, 2100, 20740])
# display(proData.iloc[doc_removed-1])

# display(proData.shape)
proData.drop(proData.index[doc_removed-1], inplace = True)
# display(proData.shape)

# display(proData.iloc[doc_removed-1])


docs = proData.Review_Text.tolist()
# display(proData.head())
# display(docs[:5])

## Computing Topic Distribution 

In [29]:
dist_top = []
topic_label = []
doc_num = []

doc_len = len(df.document.unique())
for i in range(1, doc_len+1):
    doc = df[df.document == i] 
    vals = doc.gamma.values.tolist()
    dist_top += [np.asarray(vals)]
    topic_label += [np.asarray(vals).argmax() + 1] 
    doc_num += [i]
    
print(dist_top)
print(topic_label)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [30]:
print(dist_top[:5])
print(topic_label[:5])
print(np.unique(np.asarray(topic_label)))

[array([0.00791124, 0.02859694, 0.01061477, 0.01111885, 0.01252749,
       0.05189843, 0.01823913, 0.02148178, 0.04950913, 0.01441668,
       0.01158759, 0.00799081, 0.00908102, 0.00386291, 0.2359392 ,
       0.27449742, 0.00983515, 0.05204575, 0.00699135, 0.02816108,
       0.01457037, 0.06296118, 0.01123746, 0.01173744, 0.02269851,
       0.01048833]), array([0.01557659, 0.06757905, 0.01683752, 0.01193885, 0.01681607,
       0.01334166, 0.02559016, 0.02352876, 0.03318149, 0.04467125,
       0.01824104, 0.12678958, 0.05923894, 0.24028776, 0.01696796,
       0.01552021, 0.01476387, 0.01372621, 0.01853611, 0.03114467,
       0.02311226, 0.01891529, 0.01439783, 0.04695941, 0.04239412,
       0.02994336]), array([0.01264254, 0.03722124, 0.02567034, 0.02172944, 0.02330691,
       0.0204117 , 0.03333349, 0.02542648, 0.34461551, 0.01245591,
       0.0209141 , 0.01928816, 0.01880726, 0.01072549, 0.0401686 ,
       0.01945983, 0.00807854, 0.02429411, 0.02009513, 0.02044043,
       0.06543308, 

In [31]:
# top_dist = []
# for doc in dist_top: 
#     top_dist.append(list(doc))
# # display(top_dist)
# print(top_dist[:5])
top_dist = dist_top

In [36]:
## Pruning documents that do not have great confidence in assigning the topic
# threshold = 1/num_topics
# _idx = np.amax(top_dist, axis=1) > threshold
# print(_idx)
# X_topic = top_dist[_idx]
# print(len(X_topic))
# print(X_topic.shape)
# print(X_topic)
X_topic = top_dist

In [33]:
print(len(X_topic))
print(len(X_topic[0]))

33620
26


In [34]:
# Clustering
from sklearn.manifold import TSNE
# tsne = TSNE(n_components=2)
tsne = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')
X_tsne = tsne.fit_transform(X_topic)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 33620 samples in 0.040s...
[t-SNE] Computed neighbors for 33620 samples in 21.901s...
[t-SNE] Computed conditional probabilities for sample 1000 / 33620
[t-SNE] Computed conditional probabilities for sample 2000 / 33620
[t-SNE] Computed conditional probabilities for sample 3000 / 33620
[t-SNE] Computed conditional probabilities for sample 4000 / 33620
[t-SNE] Computed conditional probabilities for sample 5000 / 33620
[t-SNE] Computed conditional probabilities for sample 6000 / 33620
[t-SNE] Computed conditional probabilities for sample 7000 / 33620
[t-SNE] Computed conditional probabilities for sample 8000 / 33620
[t-SNE] Computed conditional probabilities for sample 9000 / 33620
[t-SNE] Computed conditional probabilities for sample 10000 / 33620
[t-SNE] Computed conditional probabilities for sample 11000 / 33620
[t-SNE] Computed conditional probabilities for sample 12000 / 33620
[t-SNE] Computed conditional probabilities for sa

In [35]:
# Generate random colors

import random
get_colors = lambda n: list(map(lambda i: "#" + "%06x" % random.randint(0, 0xFFFFFF),range(n)))
colormap = np.asarray(get_colors(num_topics)) 
print(colormap)

['#21584c' '#00751c' '#0cf769' '#e3b9c5' '#d8eb50' '#589ebd' '#3376b5'
 '#0c250d' '#426ac0' '#d0e8d8' '#bce6a1' '#50a7e8' '#c3b98d' '#817b1d'
 '#f3c95f' '#a9a3e7' '#a755a2' '#4762e3' '#f681d7' '#6daa71' '#ce7d97'
 '#c4554e' '#4215a1' '#7fb6ee' '#e4d25c' '#49d83a']


In [103]:
df_cluster = pd.DataFrame()
df_cluster['X_tsne'] = X_tsne[:, 0]
df_cluster['Y_tsne'] = X_tsne[:, 1]
df_cluster['Topic'] = topic_label
df_cluster['Doc_Num'] = doc_num
df_cluster['Document'] = docs

cluster_colors = dict()
for c in range(num_topics): 
    cluster_colors[c] = colormap[c]
    
df_cluster['Colors'] = df_cluster['Topic'].apply(lambda l: cluster_colors[l])
df_cluster.sort_values(by=['Topic'], inplace=True)

display(df_cluster.head())

,X_tsne,Y_tsne,Topic,Document,Colors
12676,-50.642681,6.352161,1,12677,#256dee
22659,-51.552792,16.849981,1,22660,#256dee
8499,-55.796822,12.443072,1,8500,#256dee
22667,-58.539524,14.939828,1,22668,#256dee
33044,-49.547268,11.914720,1,33045,#256dee


In [104]:
df_cluster.Topic.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25])

In [84]:
from bokeh.plotting import figure, show, output_notebook, save#, output_file
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource
output_notebook()

Loading BokehJS ...

In [109]:
source = ColumnDataSource(dict(
    x=df_cluster['X_tsne'],
    y=df_cluster['Y_tsne'],
    color=df_cluster['Colors'],
    label=df_cluster['Topic'],
    doc_num=df_cluster['Doc_Num']
    document = df_cluster["Document"]
#     msize= p_df['marker_size'],
#     topic_key= p_df['clusters'],
#     content = p_df['tokenz'],
#     review_text = p_df['Review_Text']
))

In [110]:
title = 'T-SNE visualization of topics'

plot_lda = figure(plot_width=1000, plot_height=700,
                     title=title, tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
                     x_axis_type=None, y_axis_type=None, min_border=1)

plot_lda.scatter(x='x', y='y',  legend='label', source=source,
                 color='color', alpha=0.8)#'msize', )

# hover tools
hover = plot_lda.select(dict(type=HoverTool))
hover.tooltips = { "Document": "@doc_num: @document  <br> Topic_num: @label "}
plot_lda.legend.location = "top_left"

show(plot_lda)

# #save the plot
# # img_path = root + '/Desktop/workspace/indeed/Job-Satisfaction/img/companies/clustering/cons'
# save(plot_lda, root+'/Desktop/Paper/img/con/24/cluster.html')